In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load the dataset (we'll use the test split for evaluation)
dataset = load_dataset('zeroshot/twitter-financial-news-sentiment', split='validation')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sent_train.csv:   0%|          | 0.00/859k [00:00<?, ?B/s]

sent_valid.csv:   0%|          | 0.00/217k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [ ]:
# Shuffle the dataset and slice it
sliced_dataset = dataset.shuffle(seed=42).select(range(1000))

In [ ]:
sliced_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

In [ ]:
# Sentiment mapping: 0 = negative, 1 = positive, 2 = neutral
sentiment_mapping = {
    0: "negative",
    1: "positive",
    2: "neutral"
}

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

In [ ]:
new_model = "Llama-2-7b-chat-finetuned"

In [ ]:
# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Ind_Proj/large_language_models

LLama_2_7b_chat_multidataset_finetuned


In [ ]:
!cp -r /content/drive/MyDrive/Ind_Proj/large_language_models/LLama_2_7b_chat_multidataset_finetuned /content/Llama-2-7b-chat-finetuned

In [ ]:
!ls /content/Llama-2-7b-chat-finetuned

adapter_config.json  adapter_model.bin	README.md


In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# Define the text-generation pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
# Function to generate sentiment
def generate_sentiment(text):
    # Construct the prompt
    prompt = f"Determine the sentiment of the financial news as negative, neutral or positive: {text}"

    # Run the text generation
    result = pipe(f"<s>[INST] {prompt} [/INST]")

    # Extract the generated text
    generated_text = result[0]['generated_text']

    # Extract the text after [/INST]
    return extract_after_inst(generated_text)

In [ ]:
# Function to extract the part after [/INST]
def extract_after_inst(generated_text):
    parts = generated_text.split('[/INST]')
    return parts[1].strip() if len(parts) > 1 else generated_text

In [ ]:
# Function to check sentiment in the model's output
def check_sentiment(output):
    output = output.lower()
    if "negative" in output:
        return "negative"
    elif "neutral" in output:
        return "neutral"
    elif "positive" in output:
        return "positive"
    else:
        return "neutral"  # Default to "neutral" if no keyword found

In [ ]:
def generate_and_classify_batch(batch):
    texts = batch['text']
    actual_labels = [sentiment_mapping[label] for label in batch['label']]

    # Generate predictions for the batch
    generated_texts = pipe([f"<s>[INST] Determine the sentiment of the financial news as negative, neutral or positive: {text} [/INST]" for text in texts])

    # Flattening the nested lists of generated texts
    if isinstance(generated_texts[0], list):
        generated_texts = [item for sublist in generated_texts for item in sublist]

    # Extract and classify sentiment from generated texts
    predicted_sentiments = [check_sentiment(extract_after_inst(result['generated_text'])) for result in generated_texts]

    print("Batch completed.")

    return {'y_true': actual_labels, 'y_pred': predicted_sentiments}

# Apply the transformation using map with batch_size
batch_size = 64
result = sliced_dataset.map(generate_and_classify_batch, batched=True, batch_size=batch_size)

# Extract true and predicted labels for evaluation
y_true = result['y_true']
y_pred = result['y_pred']

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.
Batch completed.


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

In [ ]:
# Evaluate the model using accuracy and F1 score
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8780
F1 Score: 0.8764
